## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped = application_df.drop(['EIN', 'NAME'], axis=1)

In [5]:
# Determine the number of unique values in each column
unique_counts = application_df.nunique()
print(unique_counts)


EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [6]:
# Look at APPLICATION_TYPE value counts
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [7]:
# Determine the cutoff value based on the count of unique application types
cutoff_value = 500  # Adjust this value as needed based on your analysis

# Create a list of application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [8]:
# Look at CLASSIFICATION value counts
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [9]:
# Filter CLASSIFICATION value counts for counts > 1
classification_counts_gt1 = classification_counts[classification_counts > 1]
print(classification_counts_gt1)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [10]:
# Determine the cutoff value based on the count of unique classifications
cutoff_value_cls = 100  # Adjust this value as needed based on your analysis

# Create a list of classifications to be replaced with "Other"
classifications_to_replace = classification_counts[classification_counts < cutoff_value_cls].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64


In [11]:
# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df)

# Display the encoded DataFrame
print(application_df_encoded.head())


        EIN  STATUS  ASK_AMT  IS_SUCCESSFUL  \
0  10520599       1     5000              1   
1  10531628       1   108590              1   
2  10547893       1     5000              0   
3  10553066       1     6692              1   
4  10556103       1   142590              1   

   NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC  \
0                                              False                
1                                              False                
2                                              False                
3                                              False                
4                                              False                

   NAME_100 BLACK MEN OF AMERICA  NAME_100 BLACK MEN OF MEMPHIS INC  \
0                          False                              False   
1                          False                              False   
2                          False                              False   
3                 

In [12]:
from sklearn.model_selection import train_test_split

# Split the preprocessed data into features (X) and target (y) arrays
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = application_df_encoded['IS_SUCCESSFUL']

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Display the shapes of the training and testing datasets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (25724, 19618)
Shape of X_test: (8575, 19618)
Shape of y_train: (25724,)
Shape of y_test: (8575,)


In [7]:
# Import necessary libraries
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read in the data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Preprocessing
# Drop unnecessary columns
application_df = application_df.drop(columns=["EIN", "NAME"])

# Determine the cutoff value for binning
cutoff_value = 500

# Look at value counts for APPLICATION_TYPE
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()

# Determine application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace application types in dataframe
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

# Look at value counts for CLASSIFICATION
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Determine classifications to be replaced with "Other"
classification_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace classifications in dataframe
for cls in classification_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df)

# Split data into features (X) and target (y) arrays
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"])
y = application_df_encoded["IS_SUCCESSFUL"]

# Split data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, input_dim=input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                450       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 461 (1.80 KB)
Trainable params: 461 (1.80 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Compile, Train and Evaluate the Model

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model - deep neural net
nn = Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
nn.add(Dense(units=10, input_dim=input_features, activation='relu'))

# Second hidden layer (optional, add if needed)
# nn.add(Dense(units=10, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                450       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 461 (1.80 KB)
Trainable params: 461 (1.80 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [10]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6230 - accuracy: 0.6939
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5738 - accuracy: 0.7276
Epoch 3/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5670 - accuracy: 0.7290
Epoch 4/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5617 - accuracy: 0.7293
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5557 - accuracy: 0.7289
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5540 - accuracy: 0.7295
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5522 - accuracy: 0.7304
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5519 - accuracy: 0.7304
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accura

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Print the model's loss and accuracy
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 2s - loss: 0.5568 - accuracy: 0.7244 - 2s/epoch - 6ms/step
Loss: 0.5567816495895386, Accuracy: 0.7244315147399902


In [13]:
# Define the file path for the exported model
model_path = "AlphabetSoupCharity.h5"

# Export the model to HDF5 file
nn.save(model_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
